In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Get the AIC of the hypothetical material from Matminer
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from pymatgen.core import Composition

feature_calculators = MultipleFeaturizer([cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])
feature_labels = feature_calculators.feature_labels()
df12['composition'] = df12['formula'].map(Composition)
df12_features = feature_calculators.featurize_dataframe(df12,col_id='composition')
df_6w_AIC = df12_features[['formula','avg ionic char']]

Through the clustering algorithm, the bandgap results of CrabNet_s are screened to find out the most suitable bandgap

In [ ]:
from sklearn.cluster import KMeans
from time import time

kmeans = KMeans(n_clusters=2, random_state=0)
def exclude_diff_hsedata(tmp:pd.Series):
    if np.sum(np.array(tmp) == 0) >= 4:
        return 0
    else:
        kmeans.fit(np.array(tmp).reshape(-1,1))
        labels = kmeans.labels_
        res = pd.DataFrame({'value': tmp, 'label': labels})
        class0 = res[res['label']==0]
        class1 = res[res['label']==1]
        if len(class0) > len(class1):
            return class0['value'].mean()
        else:
            return class1['value'].mean()

start = time()

#df_6w_AIC is 6.6w data with AIC characteristics, and the fold is the df_6w_AIC data prefix with [['hse_fold0','hse_fold1','hse_fold2','hse_fold3','hse_fold4']]
fold = df_6w_AIC[['hse_fold0','hse_fold1','hse_fold2','hse_fold3','hse_fold4']]
df_6w_AIC['mean'] = fold.iloc[:,:].apply(lambda x: exclude_diff_hsedata(x), axis=1)
end = time()
print(end - start)

#"New_Eg_6.6w.csv" is given directly in the Data file
df_6w_AIC.to_csv('./Data/New_Eg_6.6w.csv',index= False)